In [ ]:
import pandas as pd
import glob
import textgrids

In [ ]:
# declare the path where the textgrids are stored
path = r'../output/textgrid/*.Textgrid'
# all files in the declared folder that end with .Textgrid are stored in a list
files = glob.glob(path)

# declare the name of the tier that you want to extract
#tier_name = 'silences' # e.g. for praat silences 
tier_name = 'word' # for whisperx

Transform all textgrids to csv and merge into one big csv
Exclude the word "een" and expletives; and check whether the last produced word was later than 30s into the one-minute trial

In [ ]:
merged_data = pd.DataFrame()

# iterates through all files in the list 
for file_name in files:

    print(file_name)
    # get the textgrid information from the file, stored in a dictionary called grid
    grid = textgrids.TextGrid(file_name)
    
    items = []
    starts = []
    durations = []
    subsequent_RT = []
    participant_id = grid.filename.split("\\")[1].split('.')[0]
    corrupted_audio = []

    # iterates through each line in the tier of interest and adds a word and its start time to lists
    for item in grid[tier_name]:
        # only consider words (not silent pauses or articles like "een")
        if (item.text !="" and item.text != "een" and item.text != "fuck"):
            items.append(item.text)
            starts.append(item.xmin)

    # calculate the durations between the start times of subsequent words
    for i in range(len(starts)):
        if i == (len(starts) - 1):
            durations.append(60 - starts[i])
        else: 
            durations.append(starts[i+1] - starts[i])
        if i == 0:
            subsequent_RT.append(0)
        else:
            subsequent_RT.append(starts[i] - starts[0])



    # check whether the trial was appropraitely long (60s), if not, do not add it to the merged csv file
    if item.xmin > 30:
        # all list are stored in a dictionary and then transformed to a csv
        dict = {'id': participant_id, 'item': items, 'start': starts, 'dur': durations, "sub_RT": subsequent_RT}
        df = pd.DataFrame.from_dict(dict)
        df.to_csv(grid.filename.split("\\")[0] + '/' + participant_id + ".csv", index_label='item_number')
        
        # concatenate all df such that there is one big data file in the end
        merged_data = pd.concat([merged_data, df])
    else:
        # add to list that contains all corrupted audio files
        corrupted_audio.append(file_name)

merged_data.to_csv(grid.filename.split("\\")[0] + '/' +'merged_CSVs.csv', index=False)

# get a csv file that lists all audio files that are not the appropriate 60s long
dictCorrFiles = {'filename': corrupted_audio}
dfCorrFiles = pd.DataFrame.from_dict(dictCorrFiles)
dfCorrFiles.to_csv('corruptedAudioFiles.csv')
print('Number of corrupted files: ' , len(corrupted_audio) )
